# Data voorbewerking

De data voor dit project is afkomstig van het back office systeem van Financial Markets, deels aangevuld met extra data vanuit Bloomberg.
De data bestaat uit de volgende elementen:

- Bond data
- Bondprijzen
- Government Yield curves
- Inflation data

Alle data is extracted en opgeslagen in csv files. In dit workbook lopen we door de data voorbereiding heen. Alle hier genoemde stappen kunnen ook geautomatiseerd worden uitgevoerd door het shell command 'Make Data'.

In [308]:
%load_ext autoreload
%autoreload 2

import sys
import pandas as pd

sys.path.insert(0, "..") 
from src.data import make_dataset
from src.features import build_features

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Bond data

In [270]:

# Get bond data, drop unneeded columns, convert formats and strip training blanks
df_bonds = make_dataset.get_bond_data()

2021-12-31 12:20:33.113 | INFO     | src.data.make_dataset:get_bond_data:34 - Load bond data
2021-12-31 12:20:33.114 | INFO     | src.data.make_dataset:read_csv:19 - Loading data from ..\data\raw\bonds.csv


In [271]:
df_bonds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 225 entries, 0 to 224
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   ccy                225 non-null    string        
 1   country            225 non-null    string        
 2   issue_dt           225 non-null    datetime64[ns]
 3   first_coupon_date  202 non-null    datetime64[ns]
 4   mature_dt          225 non-null    datetime64[ns]
 5   isin               225 non-null    string        
 6   issuer_name        225 non-null    string        
 7   coupon_frq         225 non-null    string        
 8   coupon             225 non-null    float64       
 9   tot_issue          225 non-null    float64       
 10  cfi_code           218 non-null    string        
 11  issue_rating       220 non-null    string        
dtypes: datetime64[ns](3), float64(2), string(7)
memory usage: 21.2 KB


In [272]:
df_bonds.head()

,ccy,country,issue_dt,first_coupon_date,mature_dt,isin,issuer_name,coupon_frq,coupon,tot_issue,cfi_code,issue_rating
0,EUR,Netherlands,2009-02-13,2009-07-15,2019-07-15,NL0009086115,STAAT DER NEDERLANDEN,ANNUAL,4.000,5.000000e+09,DBFTFN,AAA
1,NLG,Austria,1994-02-28,1995-02-28,2024-02-28,NL0000133924,AUSTRIA,ANNUAL,6.250,1.000000e+09,DBFTXB,AA+
2,EUR,Netherlands,2012-03-09,2013-01-15,2033-01-15,NL0010071189,STAAT DER NEDERLANDEN,ANNUAL,2.500,4.160000e+09,DBFXXN,AAA
3,USD,United States,2009-05-15,2009-11-15,2019-05-15,US912828KQ20,UNITED STATES TREASURY,SEMI ANNUAL,3.125,6.441100e+10,<NA>,<NA>
4,USD,United States,2010-02-15,2010-08-15,2020-02-15,US912828MP29,UNITED STATES TREASURY,SEMI ANNUAL,3.625,0.000000e+00,<NA>,<NA>


Imputeren ontbrekende waarden voor issue rating. 
Issue rating wordt waar deze ontbreekt ingevuld met de meest voorkomende issue rating voor de issuer.

CFI code wordt waar deze ontbreekt ingevuld met code 'onbekend' = DXXXXX.

Datums met de waarde 1899-12-30 zijn default waardes van het bronsysteem. Deze worden verwijderd.
Daar waar de eerste coupon datum ontbreekt (o.a. zero coupon bonds) - wordt deze aangevuld met de issue datum.
De reden hiervoor is dat we zo zonder veel moeite de looptijd van de bond kunnen berekenen.

In [273]:
df_bonds = make_dataset.impute_bonds(df_bonds)

2021-12-31 12:20:33.505 | INFO     | src.data.make_dataset:impute_bonds:67 - Impute bond data


In [274]:
make_dataset.save_pkl('bonds', df_bonds)

2021-12-31 12:20:33.615 | INFO     | src.data.make_dataset:save_pkl:287 - Save preprocessed bonds data


In [275]:
df_bonds.count()

ccy                  225
country              225
issue_dt             225
first_coupon_date    225
mature_dt            225
isin                 225
issuer_name          225
coupon_frq           225
coupon               225
tot_issue            225
cfi_code             225
issue_rating         225
bond_duration        225
dtype: int64

# Bondprijzen

In [276]:
df_price = make_dataset.get_price()

2021-12-31 12:20:33.834 | INFO     | src.data.make_dataset:get_price:104 - Load bond price data
2021-12-31 12:20:33.834 | INFO     | src.data.make_dataset:read_csv:19 - Loading data from ..\data\raw\price.csv


In [277]:
df_price = make_dataset.impute_price(df_price)

2021-12-31 12:20:34.467 | INFO     | src.data.make_dataset:impute_price:129 - Impute bond price


In [278]:
df_price.head()

,reference_identifier,ccy,rate_dt,mid
0,BE0000332412,EUR,2014-01-20,100.719
1,BE0000332412,EUR,2014-01-24,101.005
2,BE0000332412,EUR,2014-01-28,100.953
3,BE0000332412,EUR,2014-01-22,100.359
4,BE0000332412,EUR,2014-01-21,100.601


In [279]:
df_price.describe()

,mid
count,209276.000000
mean,111.289025
std,15.522802
min,84.429000
25%,102.285000
50%,106.729000
75%,112.926000
max,195.749000


In [280]:
df_price.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 209276 entries, 0 to 220236
Data columns (total 4 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   reference_identifier  209276 non-null  string        
 1   ccy                   209276 non-null  string        
 2   rate_dt               209276 non-null  datetime64[ns]
 3   mid                   209276 non-null  float64       
dtypes: datetime64[ns](1), float64(1), string(2)
memory usage: 8.0 MB


In [281]:
make_dataset.save_pkl('price', df_price)

2021-12-31 12:20:35.015 | INFO     | src.data.make_dataset:save_pkl:287 - Save preprocessed price data


# Government Yield curves

In [282]:
df_yield = make_dataset.get_yield()

2021-12-31 12:20:38.083 | INFO     | src.data.make_dataset:get_yield:137 - Load goverment yield curve data
2021-12-31 12:20:38.084 | INFO     | src.data.make_dataset:read_csv:19 - Loading data from ..\data\raw\yield.csv


In [283]:
df_yield = make_dataset.impute_yield(df_yield)

2021-12-31 12:20:39.407 | INFO     | src.data.make_dataset:impute_yield:166 - Impute yield curve


In [284]:
df_yield.head()

,country,rate_dt,timeband,ratename,ccy,actual_dt,datedays,bid,offer,int_basis
1,France,2016-06-01,1 YEAR,GOV Yield Curve FR BB,EUR,2017-06-05,369,-0.5200,-0.498,ANNUAL
2,France,2016-06-01,10 YEARS,GOV Yield Curve FR BB,EUR,2026-06-03,3654,0.4755,0.475,ANNUAL
3,France,2016-06-01,15 YEARS,GOV Yield Curve FR BB,EUR,2031-06-03,5480,0.4755,0.475,ANNUAL
4,France,2016-06-01,2 YEARS,GOV Yield Curve FR BB,EUR,2018-06-04,733,-0.4597,-0.442,ANNUAL
5,France,2016-06-01,20 YEARS,GOV Yield Curve FR BB,EUR,2036-06-03,7307,1.2151,1.220,ANNUAL


In [285]:
df_yield.describe()

,datedays,bid,offer
count,89750.000000,89750.000000,89750.000000
mean,3389.135610,0.284506,0.273997
std,2877.657017,0.921237,0.921592
min,367.000000,-1.038000,-1.051000
25%,1463.000000,-0.429000,-0.443000
50%,2560.000000,0.016000,0.009000
75%,3657.000000,0.756000,0.750000
max,10968.000000,4.128000,4.109000


In [286]:
make_dataset.save_pkl('yield', df_yield)

2021-12-31 12:20:39.814 | INFO     | src.data.make_dataset:save_pkl:287 - Save preprocessed yield data


# Inflation data


In [287]:
df_inflation = make_dataset.get_inflation()  

2021-12-31 12:20:42.589 | INFO     | src.data.make_dataset:get_inflation:178 - Load goverment yield curve data
2021-12-31 12:20:42.591 | INFO     | src.data.make_dataset:read_csv:19 - Loading data from ..\data\raw\DE Inflation.csv
2021-12-31 12:20:42.608 | INFO     | src.data.make_dataset:read_csv:19 - Loading data from ..\data\raw\FR Inflation.csv
2021-12-31 12:20:42.625 | INFO     | src.data.make_dataset:read_csv:19 - Loading data from ..\data\raw\ES Inflation.csv
2021-12-31 12:20:42.646 | INFO     | src.data.make_dataset:read_csv:19 - Loading data from ..\data\raw\IT Inflation.csv
2021-12-31 12:20:42.664 | INFO     | src.data.make_dataset:read_csv:19 - Loading data from ..\data\raw\US Inflation.csv


In [288]:
df_inflation = make_dataset.impute_inflation(df_inflation)


df_inflation.info()

2021-12-31 12:20:57.454 | INFO     | src.data.make_dataset:impute_inflation:205 - Impute inflation curve


<class 'pandas.core.frame.DataFrame'>
Int64Index: 215007 entries, 0 to 216873
Data columns (total 5 columns):
 #   Column     Non-Null Count   Dtype         
---  ------     --------------   -----         
 0   country    215007 non-null  string        
 1   rate_dt    215007 non-null  datetime64[ns]
 2   timeband   215007 non-null  string        
 3   inflation  215007 non-null  float64       
 4   ratename   215007 non-null  string        
dtypes: datetime64[ns](1), float64(1), string(3)
memory usage: 9.8 MB


In [289]:
make_dataset.save_pkl('inflation', df_inflation)

2021-12-31 12:20:57.613 | INFO     | src.data.make_dataset:save_pkl:287 - Save preprocessed inflation data


In [311]:
df_yield_pivot = pd.pivot(df_yield, index = ['country','rate_dt'], columns = ['timeband'], values = ['bid','offer'])

In [295]:
df_yield_pivot = pd.pivot(df_yield, columns = ['timeband'], values = ['bid','offer'])

In [296]:
df_bpi = make_dataset.join_inflation(df_bonds, df_price, df_inflation)

In [299]:
df_bpy = make_dataset.join_yield(df_bonds, df_price, df_yield)

In [426]:
df_bpiy = make_dataset.fulljoin(df_bonds, df_price, df_inflation, df_yield)

TypeError: pivot() got an unexpected keyword argument 'valuenames'

In [409]:
df_bpiy = build_features.add_duration(df_bpiy)


In [410]:
df_bpiy.columns

Index(['reference_identifier', 'ccy', 'rate_dt', 'mid', 'country', 'issue_dt',
       'first_coupon_date', 'mature_dt', 'isin', 'issuer_name', 'coupon_frq',
       'coupon', 'tot_issue', 'cfi_code', 'issue_rating', 'bond_duration',
       'inflation_1', 'inflation_2', 'inflation_3', 'inflation_4',
       'inflation_5', 'inflation_6', 'inflation_7', 'inflation_8',
       'inflation_9', 'inflation_10', 'inflation_15', 'inflation_20',
       'inflation_25', 'inflation_30', 'yield_bid1', 'yield_offer1',
       'yield_bid2', 'yield_offer2', 'yield_bid3', 'yield_offer3',
       'yield_bid4', 'yield_offer4', 'yield_bid5', 'yield_offer5',
       'yield_bid6', 'yield_offer6', 'yield_bid7', 'yield_offer7',
       'yield_bid8', 'yield_offer8', 'yield_bid9', 'yield_offer9',
       'yield_bid10', 'yield_offer10', 'yield_bid15', 'yield_offer15',
       'yield_bid20', 'yield_offer20', 'yield_bid30', 'yield_offer30',
       'remain_duration'],
      dtype='object')

In [415]:
df_bpiye = build_features.add_bid_offer_spread(df_bpiy)



KeyError: '1'

In [440]:
df_bpiy.columns

Index(['reference_identifier', 'ccy', 'rate_dt', 'mid', 'country', 'issue_dt',
       'first_coupon_date', 'mature_dt', 'isin', 'issuer_name', 'coupon_frq',
       'coupon', 'tot_issue', 'cfi_code', 'issue_rating', 'bond_duration',
       'inflation_1', 'inflation_2', 'inflation_3', 'inflation_4',
       'inflation_5', 'inflation_6', 'inflation_7', 'inflation_8',
       'inflation_9', 'inflation_10', 'inflation_15', 'inflation_20',
       'inflation_25', 'inflation_30', 'yield_bid1', 'yield_offer1',
       'yield_bid2', 'yield_offer2', 'yield_bid3', 'yield_offer3',
       'yield_bid4', 'yield_offer4', 'yield_bid5', 'yield_offer5',
       'yield_bid6', 'yield_offer6', 'yield_bid7', 'yield_offer7',
       'yield_bid8', 'yield_offer8', 'yield_bid9', 'yield_offer9',
       'yield_bid10', 'yield_offer10', 'yield_bid15', 'yield_offer15',
       'yield_bid20', 'yield_offer20', 'yield_bid30', 'yield_offer30',
       'remain_duration'],
      dtype='object')

In [441]:
df_bidofferspread = build_features.add_bidoffer_spread(df_bpiy)

In [447]:
df_bpiye = pd.concat([df_bpiy, df_bidofferspread], axis=1)

In [448]:
len(df_bidofferspread)

67636

In [450]:
df_bpiye.head()

,reference_identifier,ccy,rate_dt,mid,country,issue_dt,first_coupon_date,mature_dt,isin,issuer_name,...,yield_offer9,yield_bid10,yield_offer10,yield_bid15,yield_offer15,yield_bid20,yield_offer20,yield_bid30,yield_offer30,remain_duration
5498,DE0001102358,EUR,2016-06-01,112.894,Germany,2014-05-23,2015-05-15,2024-05-15,DE0001102358,Bondsrep ber,...,0.06,0.131,0.134,0.131,0.134,0.6485,0.656,0.8331,0.84,2905 days
22988,DE0001102374,EUR,2016-06-01,104.438,Germany,2015-01-16,2016-02-15,2025-02-15,DE0001102374,Bondsrep ber,...,0.06,0.131,0.134,0.131,0.134,0.6485,0.656,0.8331,0.84,3181 days
35196,DE0001141729,EUR,2016-06-01,103.045,Germany,2015-07-03,2016-10-16,2020-10-16,DE0001141729,Bondsrep ber,...,0.06,0.131,0.134,0.131,0.134,0.6485,0.656,0.8331,0.84,1598 days
35198,DE0001102382,EUR,2016-06-01,108.599,Germany,2015-07-17,2016-08-15,2025-08-15,DE0001102382,Bondsrep ber,...,0.06,0.131,0.134,0.131,0.134,0.6485,0.656,0.8331,0.84,3362 days
35200,DE0001102390,EUR,2016-06-01,103.510,Germany,2016-01-15,2017-02-15,2026-02-15,DE0001102390,Bondsrep ber,...,0.06,0.131,0.134,0.131,0.134,0.6485,0.656,0.8331,0.84,3546 days


In [437]:
df_spread

,yield_spread_1,yield_spread_2,yield_spread_3,yield_spread_4,yield_spread_5,yield_spread_6,yield_spread_7,yield_spread_8,yield_spread_9,yield_spread_10,yield_spread_15,yield_spread_20,yield_spread_30
0,-1.098,-1.039,-1.0153,-0.937,-0.769,-0.5971,-0.4558,-0.2356,0.1171,0.265,0.265,1.3045,1.6731
1,-1.098,-1.039,-1.0153,-0.937,-0.769,-0.5971,-0.4558,-0.2356,0.1171,0.265,0.265,1.3045,1.6731
2,-1.098,-1.039,-1.0153,-0.937,-0.769,-0.5971,-0.4558,-0.2356,0.1171,0.265,0.265,1.3045,1.6731
3,-1.098,-1.039,-1.0153,-0.937,-0.769,-0.5971,-0.4558,-0.2356,0.1171,0.265,0.265,1.3045,1.6731
4,-1.098,-1.039,-1.0153,-0.937,-0.769,-0.5971,-0.4558,-0.2356,0.1171,0.265,0.265,1.3045,1.6731
...,...,...,...,...,...,...,...,...,...,...,...,...,...
67631,-0.945,-0.885,-0.7890,-0.561,-0.259,-0.1040,0.1070,0.3100,0.5090,0.781,1.597,1.6250,2.5480
67632,-0.945,-0.885,-0.7890,-0.561,-0.259,-0.1040,0.1070,0.3100,0.5090,0.781,1.597,1.6250,2.5480
67633,-0.930,-0.896,-0.7810,-0.549,-0.228,-0.0600,0.1450,0.3380,0.5390,0.815,1.637,1.6640,2.5980
67634,-0.930,-0.896,-0.7810,-0.549,-0.228,-0.0600,0.1450,0.3380,0.5390,0.815,1.637,1.6640,2.5980


In [403]:
df_bpiye.columns

Index(['reference_identifier', 'ccy', 'rate_dt', 'mid', 'country', 'issue_dt',
       'first_coupon_date', 'mature_dt', 'isin', 'issuer_name', 'coupon_frq',
       'coupon', 'tot_issue', 'cfi_code', 'issue_rating', 'bond_duration',
       'inflation_1', 'inflation_2', 'inflation_3', 'inflation_4',
       'inflation_5', 'inflation_6', 'inflation_7', 'inflation_8',
       'inflation_9', 'inflation_10', 'inflation_15', 'inflation_20',
       'inflation_25', 'inflation_30', 'yield_bid1', 'yield_offer1',
       'yield_bid2', 'yield_offer2', 'yield_bid3', 'yield_offer3',
       'yield_bid4', 'yield_offer4', 'yield_bid5', 'yield_offer5',
       'yield_bid6', 'yield_offer6', 'yield_bid7', 'yield_offer7',
       'yield_bid8', 'yield_offer8', 'yield_bid9', 'yield_offer9',
       'yield_bid10', 'yield_offer10', 'yield_bid15', 'yield_offer15',
       'yield_bid20', 'yield_offer20', 'yield_bid30', 'yield_offer30',
       'remain_duration', 'spread1', 'spread2', 'spread3', 'spread4',
       'spread5

In [380]:
df_bpiy.columns

Index(['reference_identifier', 'ccy', 'rate_dt', 'mid', 'country', 'issue_dt',
       'first_coupon_date', 'mature_dt', 'isin', 'issuer_name', 'coupon_frq',
       'coupon', 'tot_issue', 'cfi_code', 'issue_rating', 'bond_duration',
       'inflation_1', 'inflation_2', 'inflation_3', 'inflation_4',
       'inflation_5', 'inflation_6', 'inflation_7', 'inflation_8',
       'inflation_9', 'inflation_10', 'inflation_15', 'inflation_20',
       'inflation_25', 'inflation_30', 'yield_bid1', 'yield_offer1',
       'yield_bid2', 'yield_offer2', 'yield_bid3', 'yield_offer3',
       'yield_bid4', 'yield_offer4', 'yield_bid5', 'yield_offer5',
       'yield_bid6', 'yield_offer6', 'yield_bid7', 'yield_offer7',
       'yield_bid8', 'yield_offer8', 'yield_bid9', 'yield_offer9',
       'yield_bid10', 'yield_offer10', 'yield_bid15', 'yield_offer15',
       'yield_bid20', 'yield_offer20', 'yield_bid30', 'yield_offer30',
       'remain_duration'],
      dtype='object')

In [343]:
df_inflation_pivot = pd.pivot(df_inflation, index = ['country','rate_dt'], columns = ['timeband'], values = 'inflation')
[''.join(('Inflation_',col)) for col in df_inflation_pivot.columns]

['Inflation_1 YEAR',
 'Inflation_10 YEARS',
 'Inflation_15 YEARS',
 'Inflation_2 YEARS',
 'Inflation_20 YEARS',
 'Inflation_25 YEARS',
 'Inflation_3 YEARS',
 'Inflation_30 YEARS',
 'Inflation_4 YEARS',
 'Inflation_5 YEARS',
 'Inflation_6 YEARS',
 'Inflation_7 YEARS',
 'Inflation_8 YEARS',
 'Inflation_9 YEARS']

In [328]:
df_bpy.to_csv(f'../data/processed/bpy.csv')
df_bpi.to_csv(f'../data/processed/bpi.csv')
df_bpiy.to_csv(f'../data/processed/bpiy.csv')


In [ ]:
countrydict: dict = {'DE': 'Germany','FR': 'France','ES': 'Spain','IT': 'Italy','US': 'United States'}
print(countrydict.values())
print(df_bpi['country'].unique())
df_bpi[df_bpi['country'].isin(countrydict.values())]['country'].unique()

In [ ]:
df_inflation['country'].unique()

In [ ]:
df = make_dataset.join_full(df_bonds,df_price,df_yield, df_inflation)



In [ ]:
df.count()

In [ ]:
df = df_price[df_price['reference_identifier'] == 'BE0000332412']

In [ ]:
df_inflation.describe()

All in one make statement...

In [ ]:
make_dataset.make_data()